# Support Function-Based T-CDFT with Remote Manager

Notebook containing the complete workflow for a singlet excitation using T-CDFT with a mixed constraint for an active molecule in an environment. It contains the following steps:

0. Initialisation
1. Run NWChem TDDFT for all `active' molecules in gas phase to determine:
    - The transition orbitals and breakdown
    - The number of negative virtual states to be optimised for in the `active' SF basis
2. Generate SF basis for the `active' molecules, requiring 3 steps:
    - Initial calculation using the standard LS parameters
    - Calculation where the SFs are optimised to represent the virtual states
    - Reoptimisation of the kernel in the above (fixed) basis to get the final kernel and coefficients
3. Generate SF basis for the `environment' molecules, involving only 1 step (can be performed at the same time as 2):
    - Calculation using standard LS parameters
4. Setup the directory structure for fragment calculations, to be performed on the front end
5. Run S0 and T-CDFT calculations for each system, for each pure constraint involved in the final transitions
6. Combine the density kernels coming from the pure constraints using the appropriate weights, on the front end
7. Run T-CDFT using the appropriate linear combination of density kernels generated in step 5

### Step 0: Initialisation

Setup systems

In [1]:
from copy import deepcopy

# give a list of dictionaries for each system
# where we give the name of each molecule and specify whether it is active or environment
# for now only one active molecule per system is assumed, this could be generalised in future, e.g. for CT states
system_compositions = {}
system_compositions['acetone'] = [{'name': 'acetone', 'type': 'active'}]
system_compositions['acetone_water'] = [{'name': 'acetone', 'type': 'active'},
                                        {'name': 'water', 'type': 'environment'}]
system_compositions['naphthalene'] = [{'name': 'naphthalene', 'type': 'active'}]
system_compositions['naphthalene_dimer'] = [{'name': 'naphthalene', 'type': 'active'},
                                            {'name': 'naphthalene', 'type': 'environment'}]


# generate the list of active molecules from the system list
active_names = []
env_names = []
for system in system_compositions:
    for molecule_dict in system_compositions[system]:
        mol_name = molecule_dict['name']
        mol_type = molecule_dict['type']
        if mol_type == 'active':
            if mol_name not in active_names:
                active_names.append(mol_name)
        elif mol_type == 'environment':
            if mol_name not in env_names:
                env_names.append(mol_name)
        else:
            print('Error in molecule type for '+mol_name+' in '+system)

print('List of active molecules:', active_names)
print('List of environment molecules:', env_names)

List of active molecules: ['acetone', 'naphthalene']
List of environment molecules: ['water', 'naphthalene']


Define calculation parameters

In [2]:
# XC functional
xc = 'PBE'

# wavelet parameters
hgrid = 0.5
crmult = 6.0
frmult = 8.0

# SF parameters
active_basis_size = 'sp_spd'
active_rloc = 8.0

env_basis_size = 's_sp'
env_rloc = 7.0
env_tag = '1'

# NWChem parameters
nwchem_basis = 'cc-pVTZ'

Setup remote manager

In [3]:
from remotemanager import Dataset, URL
from remotemanager.connection.computers.base import BaseComputer
from remotemanager.serialisation import serialdill

# remote computer username
user = 'lra040'

# remote directory to work in 
remote_dir = f'/work/e89/e89/lra040/tcdft'

# directory to run in
calc_dir = 'tcdft_workflow'

# parameters associated with the supercomputer
url = BaseComputer.from_yaml('archer2.yaml', user=user, passfile='/home/ff21106/Documents/.shp', timeout=15)
#url = BaseComputer.from_repo('archer2', user=user, passfile='/home/lr408/.ssh/.shp', timeout=15)
url.python = 'python3'

url.extra = """
export BIGDFT_MPIRUN='srun --hint=nomultithread --distribution=block:block'
export NWCHEM_MPIRUN='srun --hint=nomultithread --distribution=block:block'
module swap PrgEnv-cray PrgEnv-gnu
module load mkl
module load nwchem
source /work/e89/e89/lra040/bigdft-suite/Build/install/bin/bigdftvars.sh
export OMP_PLACES=cores"""

url.account = 'e89-bris-c'

Read in the systems and constituent molecules as BigDFT systems, changing the atom names to include the environment tag for environment molecules - this allows us to specify different SF parameters for active and environment molecules

In [4]:
def read_molecule(molecule_name, env_tag=None):
    from os.path import join
    from BigDFT.IO import read_xyz
    
    with open(join('xyzs', molecule_name+'.xyz')) as ifile:
        sys = read_xyz(ifile)
     
    # add the environment tag if argument is present
    if env_tag is not None:
        for frag in sys:
            for at in sys[frag]:
                at.sym += env_tag       
    return sys
     
    
def read_system(system_name, system_composition, active_names, active_structures, 
                env_names, env_structures, env_tag):
    from os.path import join
    from BigDFT.IO import read_xyz
    
    with open(join('xyzs', system_name+'.xyz')) as ifile:
        sys = read_xyz(ifile)    
        
    # loop over the constituent molecules, adding the environment tag as needed
    atom_tags = []
    for m,molecule in enumerate(system_composition):
        if system_composition[m]['type']  == 'active':
            mol_ind = active_names.index(system_composition[m]['name'])
            nat = len(active_structures[mol_ind])
            atom_tags += [''] * nat
        elif system_composition[m]['type']  == 'environment':
            mol_ind = env_names.index(system_composition[m]['name'])
            nat = len(env_structures[mol_ind])
            atom_tags += [env_tag] * nat

    iat = 0
    if env_tag is not None:
        for frag in sys:
            for at in sys[frag]:
                at.sym += atom_tags[iat]
                iat += 1
    
    return sys
    

active_structures = []
for molecule in active_names:
    active_structures.append(read_molecule(molecule))
        
env_structures = []
for molecule in env_names:
    env_structures.append(read_molecule(molecule, env_tag=env_tag))
        
system_structures = []
for system in system_compositions:
    system_structures.append(read_system(system, system_compositions[system], active_names, active_structures,
                                         env_names, env_structures, env_tag))

### Step 1: TDDFT Calculation with NWChem

Define the function for treating a system with TDDFT, using PBE or PBE0

In [5]:
def run_nwchem_tddft(sys_name, sys, basis, xc, thresh=0.001, nstates=1, restricted=True, tda=True):
    from BigDFT.Interop.NWChemInterop import NWBlock, nwchem_basis, NWChemCalculator
    
    Ha2eV = 27.211396132

    # Create an input file
    inp = []

    # Create the basis set
    inp.append(nwchem_basis(sys, basis, spherical='spherical'))

    # Create the DFT block
    if xc == 'LDA':
        dftvals = ['xc slater vwn_5']
    if xc == 'PBE':
        dftvals = ['xc xpbe96 1.0 cpbe96 1.0']
    elif xc == 'PBE0':
        dftvals = ['xc pbe0']
    else:
        print('Warning, currently only LDA, PBE and PBE0 have been implemented, defaulting to PBE')
        dftvals = ['xc xpbe96 1.0 cpbe96 1.0']
        
    dftvals += ['convergence nolevelshifting', 'iterations 200', 
                'grid medium', 'direct', 'noio', 'grid nodisk']
    if not restricted:
        dftvals += ['odft']
    inp.append(NWBlock('dft', values=dftvals))

    # Create the TDDFT Block
    # Search for more roots than we actually want to be safe
    if tda:
        tddft_block = NWBlock('tddft', values=['CIS', NWBlock('nroots', nstates+5)]) 
    else:
        tddft_block = NWBlock('tddft', values=['RPA', NWBlock('nroots', nstates+5)]) 
    inp.append(tddft_block)
    inp.append(NWBlock('task', 'tddft'))

    # Create a calculator
    code = NWChemCalculator(skip=True)
    log = code.run(sys=sys, input=inp, name=sys_name)
    
    # Extract the excitation information
    energy = {}
    components = {}

    tddft_data = log.get_tddft_roots()
    
    for st in ['singlet']:
        istate = 0
        for state in tddft_data:
            if state == st:
                for istate in range(nstates):
                    comp = tddft_data[state][istate]
                    tag = state.capitalize()[0] + str(istate + 1)
                    energy[tag] = comp['Energy'] * Ha2eV
                    ihomo = log.nocca
                    components[tag] = []
                    for v in comp['Components']:
                        occ_orb = v['Occ']
                        virt_orb = v['Virt']
                        if not restricted:
                            occ_spin = v['Spin_Occ']
                            virt_spin = v['Spin_Virt']
                        else:
                            occ_spin = 0
                            virt_spin = 0
                        c = v['Comp']**2
                        if c > thresh:
                            components[tag].append([occ_orb-ihomo, virt_orb-ihomo-1, c, occ_spin, virt_spin]) 
            
    return {'energy': energy, 'components': components}

Setup and submit the dataset for the active molecules

In [6]:
from os.path import join

nnodes = 1
nmpi = 128
nomp = 1

run_args = {}
run_args['mpi'] = nnodes * nmpi
run_args['omp'] = nomp
run_args['time'] = 8 * 3600

active_tddft_runs = Dataset(function=run_nwchem_tddft, url=url, name='run_nwchem_tddft',
                            remote_dir=join(remote_dir, calc_dir), local_dir=calc_dir,
                            serialiser=serialdill(), skip=False)

active_tddft_run_names = []

for m,molecule in enumerate(active_names):  
    run_name = molecule
    run_args['jobname'] = run_name+'_TDDFT'
    func_kwargs = dict(sys_name=molecule, sys=active_structures[m], basis=nwchem_basis, xc=xc,
                       thresh=0.01, restricted=True, tda=False)
    
    active_tddft_runs.append_run(args=func_kwargs, local_dir=calc_dir, **run_args)
    active_tddft_run_names.append(run_name)    

active_tddft_runs.run()

runner runner-0 already exists
runner runner-1 already exists
assessing run for runner run_nwchem_tddft-be49c8d0-runner-0... skipping already completed run
assessing run for runner run_nwchem_tddft-be49c8d0-runner-1... skipping already completed run


Pause to allow calculations to finish running if running the notebook using "run all"

In [7]:
assert False

AssertionError: 

Check status

In [8]:
print(active_tddft_runs.is_finished)

[True, True]


Fetch results

In [9]:
active_tddft_runs.fetch_results()

Summarise the results in a table.

In [10]:
import pandas as pd
from IPython.display import display

# convert the orbital index to a string for displaying in a table
def orbital_index_to_name(index, occupied=True):
    if occupied:
        if index == 0:
            name = 'HOMO'
        else:
            name = 'HOMO'+str(index)
    else:
        if index == 0:
            name = 'LUMO'
        else:
            name = 'LUMO+'+str(index)
            
    return name

# define a function to return a table summarising the excitations for each molecule
def get_tddft_table(molecule, tddft_data, thresh=0, restricted=True):
    
    # even if we perform TDDFT calculation in unrestricted mode, we don't want a spin dependent constraint
    # since all our molecules are closed shell, we can just add a factor of 2,
    # but we might want to treat the spin components more robustly in the future
    if restricted:
        spin_fac = 1
    else:
        spin_fac = 2
    
    # find all pairs of orbitals involved in transitions above some threshold
    # note this threshold must be >= that specified when extracting the results from NWChem
    orbital_indices = []
    for state in tddft_data['components']:
        for component in tddft_data['components'][state]:
            if component[2] > thresh:
                if [component[0], component[1]] not in orbital_indices:
                    orbital_indices.append([component[0], component[1]])
    orbital_indices.sort(key=lambda tup: tup[1])
    orbital_indices.sort()

    # get the column headings based on the involved orbitals
    column_names = ['Energy']
    for ind in orbital_indices:
        occ_name = orbital_index_to_name(ind[0], occupied=True)
        virt_name = orbital_index_to_name(ind[1], occupied=False)
        column_names.append(occ_name+' -> '+virt_name)
    column_names.append('Sum')              
    
    # now the row headings, which are just the excited states
    row_names = [molecule] + [state for state in tddft_data['components']]    
    
    # fill in the table data, reordering the components as we go
    table_data = [[''] * (len(column_names))]
    for state in tddft_data['components']:
        table_row = ['{0:.2f}'.format(tddft_data['energy'][state])]
        transition_sum = 0.0
        for ind in orbital_indices:
            occ_ind = ind[0]
            virt_ind = ind[1]
            found = False
            for component in tddft_data['components'][state]:
                if component[0] == occ_ind and component[1] == virt_ind and component[2] > thresh:
                    table_row.append('{0:.3f}'.format(spin_fac * component[2]))
                    transition_sum += spin_fac * component[2]
                    found = True
                    break
            if not found:
                table_row.append('')
        table_row.append('{0:.3f}'.format(transition_sum))
        table_data.append(table_row)
            
    table = pd.DataFrame(table_data, index=row_names, columns=column_names)
    return table
    
        
# summarise the TDDFT results in a single table for each active molecule
for m,molecule in enumerate(active_names): 
    tddft_table = get_tddft_table(molecule, active_tddft_runs.results[m], restricted=True)
    display(tddft_table)

,Energy,HOMO -> LUMO,Sum
acetone,,,
S1,4.18,1.000,1.000


,Energy,HOMO-2 -> LUMO+2,HOMO-1 -> LUMO+1,HOMO -> LUMO,Sum
naphthalene,,,,,
S1,4.07,0.036,0.033,0.932,1.001


Define the number of virtual orbitals based on the highest energy virtual state involved in the target excitations, noting that if this is very high it might not be possible to represent the states sufficiently accurately in a localised support function basis

In [11]:
nvirts = {}
for m,molecule in enumerate(active_names):   
    nvirts[molecule] = 0
    for state in active_tddft_runs.results[m]['components']:
        for component in active_tddft_runs.results[m]['components'][state]:
            nvirts[molecule] = max(nvirts[molecule], component[1] + 1)

### Step 2: Generate SF Basis for Active and Environment Molecules

Define function to get name of SF radicals based on SF parameters, to save having to remember syntax. Also do the same for system calculations, which cannot be so strictly tied to the basis due to different environment and active.

In [12]:
def get_radical_name(molecule, xc, basis_size, rloc):
    return molecule+'_'+str(xc)+'_SF_'+basis_size+'_'+str(rloc)

from remotemanager import RemoteFunction

@RemoteFunction
def get_system_name(system, xc, calc_type, orbitals=None, mixed=False):
    name = system+'_'+str(xc)+'_SF_'+calc_type
    # should only be one or another, would be too long to include all orbitals in a mixed calculation
    if mixed and orbitals is not None:
        print('Error, orbitals should not be specified in the case of mixed calculations')
        assert False
    if orbitals is not None:
        name += '_pure_'+orbitals[0]+'_'+orbitals[1]
    elif mixed:
        name += '_mixed'        
    return name

Define function to set up basis set parameters for linear, virtual or system (i.e. fixed basis)

In [17]:
# input_type = linear, virtual or system
# if this is a mixed basis calculation, then all arguments related to the environment basis must be present
def get_LS_input_file(hgrid, crmult, frmult, xc, basis_size, rloc, input_type='linear', nvirt=0,
                      env_basis_size=None, env_rloc=None, env_tag=None):
    from BigDFT import Inputfiles as I, InputActions as A
    import sf_sizes
    from os.path import join, abspath, dirname
    import sys
    
    # define input file
    inp = I.Inputfile()
    
    # setup wavelet basis set and PSP parameters
    inp.set_hgrid(hgrid)
    inp.set_rmult(coarse=crmult, fine=frmult)
    inp.set_xc(xc)

    # set the PSP file - here we hard code the BigDFT path on this laptop
    # to eventually be made more robust
    path = abspath('/home/ff21106/bigdft-suite/PyBigDFT/BigDFT/Database/psppar/SS')
    #path = abspath('/home/lr408/bigdft_gl_lsim/PyBigDFT/BigDFT/Database/psppar/SS')
    for element in ['C', 'N', 'H', 'O']:
        psp_file = abspath(join(path, 'psppar.'+element+'.yaml'))
        inp.set_psp_file(filename=psp_file, element=element)
        if env_tag is not None:
            inp.set_psp_file(filename=psp_file, element=element+env_tag)
        
    # common LS parameters
    inp['import'] = 'linear'
    inp.update({'lin_general': {'rpnrm_cv': 1.0e-11, 'charge_multipoles': 0}})
        
    # linear calculation for ground state
    if input_type == 'linear':
        inp.update({'lin_general': {'output_wf': 1, 'output_mat': 1}})
        inp.update({'lin_kernel': {'linear_method': 'DIAG', 'alphamix': 0.1}})

    # non-self-consistent linear calculations for virtual state(s)
    elif input_type == 'virtual':
        inp['dft'].update({'inputpsiid': 'linear_restart'}) 
        inp.update({'perf': {'adjust_kernel_iterations': False, 'adjust_kernel_threshold': False}})
        inp['lin_general'].update({'hybrid': False, 'kernel_restart_mode': 'coeff', 'nit': [0, 200],
                                   'rpnrm_cv': 1.0e-8, 'output_wf': 1, 'output_mat': 1, 'extra_states': nvirt})
        inp.update({'lin_basis': {'nit': 4, 'idsx': 2, 'fix_basis': 0.0, 'gnrm_cv': 1.0e-2}})
        inp.update({'lin_kernel': {'nit': 1, 'rpnrm_cv': 1.0e-8, 'delta_pnrm': -1, 'linear_method': 'DIRMIN', 
                                   'alphamix': 0.0, 'nstep':  500, 'gnrm_cv_coeff': 1.0e-4}})
        
        # crashes while writing orbitals, only needed for WFN-based TCDFT or for CT parameter (not yet here)
        # so skipping for now
        #inp.update({'output': {'orbitals': 'text'}})
        inp.update({'mix': {'norbsempty': nvirt}})
    
    # the system input file, for calculations to be performed in a fixed basis
    elif input_type == 'system':
        inp['dft'].update({'inputpsiid': 'linear_restart'}) 
        inp.update({'perf': {'adjust_kernel_iterations': False, 'adjust_kernel_threshold': False}})
        inp.update({'lin_general': {'output_wf': 0, 'output_mat': 0, 'hybrid': False, 'output_fragments': 2,
                                    'nit': [0, 1], 'subspace_diag': True, 'charge_multipoles': 0}})
        inp.update({'lin_basis': {'nit': 1, 'idsx': 0}})
        inp.update({'lin_kernel': {'nit': 100, 'rpnrm_cv': 1.0e-11, 'delta_pnrm': -1, 
                                   'alphamix': 0.1, 'linear_method': 'DIAG'}})

    else:
        print('Error with input_type ', input_type, ' must be linear, virtual or system')
        assert False

    # set up the basis, making sure the kernel cutoff is large enough to give dense matrices needed by T-CDFT
    if rloc == 'l':
        rlock = 'l'
    else:
        rlock = rloc + 100
    # if we have a tag and this is not a mixed calculation, use it now
    if env_tag is not None and (env_basis_size is None or env_rloc is None):
        lin_basis_dict = sf_sizes.set_support_function_dict(rloc, rlock, basis_size, tag=env_tag)
    else:
        lin_basis_dict = sf_sizes.set_support_function_dict(rloc, rlock, basis_size)
    
    if env_tag is not None and env_basis_size is not None and env_rloc is not None:
        if env_rloc == 'l':
            env_rlock = 'l'
        else:
            # the kernel cutoff can be smaller, as we are not performing an excitation which needs dense matrices
            env_rlock = env_rloc + 10
        env_mol_lin_basis_dict = sf_sizes.set_support_function_dict(env_rloc, env_rlock,
                                                                    env_basis_size, tag=env_tag)
        lin_basis_dict['lin_basis_params'].update(env_mol_lin_basis_dict['lin_basis_params'])
        lin_basis_dict['ig_occupation'].update(env_mol_lin_basis_dict['ig_occupation'])
        
    # the tag on its own is fine, but other possibilities are meaningless
    elif env_basis_size is not None or env_rloc is not None:
        print('Error, all environment basis parameters must be present, ignoring')

    inp.update(lin_basis_dict)   

    return inp

Define a function to generate the SF basis, which will run the different calculations as needed

In [14]:
def generate_sf_basis(system, inpl=None, inpv=None, inps=None):
    from BigDFT import Calculators as C
    from futile.Utils import create_tarball
    from os.path import join
    
    Ha2eV = 27.211396132

    # create the calculator
    code = C.SystemCalculator(skip=True)

    # save a summary of the results
    results_dict = {}
     
    if inpl is not None:
        # SF calculation using standard LS parameters
        lin_name = inpl['radical']+'_linear'
        run_lin = code.run(input=inpl, sys=system, name=lin_name) 
        ihomo = run_lin.log['Total Number of Orbitals']
        results_dict['ihomo'] = ihomo
        results_dict['linear'] = {}
        results_dict['linear']['energy'] = Ha2eV * run_lin.energy / run_lin.nat
        results_dict['linear']['evals'] = [Ha2eV * e for e in run_lin.evals[0][0]]
    
    if inpv is not None:
        # SF calculation also optimising the virtual states
        virt_name = inpv['radical']+'_virtual'
        run_virt = code.run(input=inpv, sys=system, name=virt_name) 
        results_dict['virtual'] = {}
        results_dict['virtual']['energy'] = Ha2eV * run_virt.energy / run_virt.nat
        results_dict['virtual']['evals'] = [Ha2eV * e for e in run_virt.evals[0][0]]

    if inps is not None:
        # SF calculation optimising only the kernel in a fixed basis
        # modify the input file to output the matrices and SFs, which we don't want to do in the more general case
        inps['lin_general'].update({'kernel_restart_mode': 'kernel', 'output_wf': 1, 'output_mat': 1})      
        s0_name = inps['radical']+'_S0'
        run_s0 = code.run(input=inps, sys=system, name=s0_name)   
        results_dict['S0'] = {}
        results_dict['S0']['energy'] = Ha2eV * run_s0.energy / run_s0.nat
        results_dict['S0']['evals'] = [Ha2eV * e for e in run_s0.evals[0][0]]
  
    return results_dict

Set up and submit dataset for active molecules

In [18]:
# change the MPI/OpenMP layout to be more suited to BigDFT
nnodes = 1
nmpi = 16
nomp = 8

run_args = {}
run_args['mpi'] = nnodes * nmpi
run_args['omp'] = nomp
run_args['time'] = 1 * 3600

active_sf_runs = Dataset(function=generate_sf_basis, url=url, name='generate_active_sf_basis',
                         remote_dir=join(remote_dir, calc_dir), local_dir=calc_dir,
                         serialiser=serialdill(), skip=False)

active_sf_run_names = []
 
for m,molecule in enumerate(active_names):  
    # get input files to pass as arguments
    inpl = get_LS_input_file(hgrid, crmult, frmult, xc, active_basis_size, active_rloc, input_type='linear')
    inpv = get_LS_input_file(hgrid, crmult, frmult, xc, active_basis_size, active_rloc, input_type='virtual',
                             nvirt=nvirts[molecule])
    inps = get_LS_input_file(hgrid, crmult, frmult, xc, active_basis_size, active_rloc, input_type='system')
    
    radical = get_radical_name(molecule, xc, active_basis_size, active_rloc)
    for inp in [inpl, inpv, inps]:
        inp.update({'radical': radical})
    run_args['jobname'] = radical
    func_kwargs = dict(system=active_structures[m], inpl=inpl, inpv=inpv, inps=inps)
    active_sf_runs.append_run(args=func_kwargs, local_dir=calc_dir, **run_args)
    active_sf_run_names.append(radical)    

active_sf_runs.run()

appended run runner-0
appended run runner-1
assessing run for runner generate_active_sf_basis-f665e94b-runner-0... running
128 total cores requested
appending partition = qos (standard)
script parsing complete
assessing run for runner generate_active_sf_basis-f665e94b-runner-1... running
128 total cores requested
appending partition = qos (standard)
script parsing complete


Do the same for environment molecules

In [20]:
run_args = {}
run_args['mpi'] = nnodes * nmpi
run_args['omp'] = nomp
run_args['time'] = 1 * 3600

env_sf_runs = Dataset(function=generate_sf_basis, url=url, name='generate_env_sf_basis',
                      remote_dir=join(remote_dir, calc_dir), local_dir=calc_dir, serialiser=serialdill(),
                      skip=False)

env_sf_run_names = []

for m,molecule in enumerate(env_names):  
    inpl = get_LS_input_file(hgrid, crmult, frmult, xc, env_basis_size, env_rloc, env_tag=env_tag,
                             input_type='linear')
    
    radical = get_radical_name(molecule, xc, env_basis_size, env_rloc)
    inpl.update({'radical': radical})
    run_args['jobname'] = radical
    func_kwargs = dict(system=env_structures[m], inpl=inpl)
    env_sf_runs.append_run(args=func_kwargs, local_dir=calc_dir, **run_args)
    env_sf_run_names.append(radical)

env_sf_runs.run()

appended run runner-0
appended run runner-1
assessing run for runner generate_env_sf_basis-e874aebb-runner-0... running
128 total cores requested
appending partition = qos (standard)
script parsing complete
assessing run for runner generate_env_sf_basis-e874aebb-runner-1... running
128 total cores requested
appending partition = qos (standard)
script parsing complete


Pause to allow calculations to finish running if running the notebook using "run all"

In [21]:
assert False

AssertionError: 

Check status

In [22]:
print(active_sf_runs.is_finished)
print(env_sf_runs.is_finished)

[True, True]
[True, True]


Fetch results

In [23]:
active_sf_runs.fetch_results()
env_sf_runs.fetch_results()

Summarise results

In [24]:
column_names = ['$E$ (eV/atom)', 'HOMO (eV)', 'LUMO (eV)']

row_names = ['Active']
table_data = [[''] * 3]
for a,molecule in enumerate(active_names):
    ihomo = active_sf_runs.results[a]['ihomo']
    for calc in ['linear', 'S0']:
        if calc == 'linear':
            row_names.append(molecule + '/' + calc)
        else:
            row_names.append(molecule + '/linear+virtual')
        energy = '{0:.3f}'.format(active_sf_runs.results[a][calc]['energy'])
        homo = '{0:.2f}'.format(active_sf_runs.results[a][calc]['evals'][ihomo - 1])
        lumo = '{0:.2f}'.format(active_sf_runs.results[a][calc]['evals'][ihomo])
        table_data.append([energy, homo, lumo])

row_names.append('Environment')
table_data.append([''] * 3)
for a,molecule in enumerate(env_names):
    row_names.append(molecule + '/linear')
    ihomo = env_sf_runs.results[a]['ihomo']
    energy = '{0:.3f}'.format(env_sf_runs.results[a]['linear']['energy'])
    homo = '{0:.2f}'.format(env_sf_runs.results[a]['linear']['evals'][ihomo - 1])
    table_data.append([energy, homo, ''])

sf_table = pd.DataFrame(table_data, index=row_names, columns=column_names)
display(sf_table)

,$E$ (eV/atom),HOMO (eV),LUMO (eV)
Active,,,
acetone/linear,-108.037,-5.53,-1.20
acetone/linear+virtual,-108.037,-5.53,-1.63
naphthalene/linear,-106.580,-5.45,-1.75
naphthalene/linear+virtual,-106.579,-5.50,-2.09
Environment,,,
water/linear,-159.672,-7.12,
naphthalene/linear,-106.575,-5.40,


### Step 4: Directory Manipulation

Define functions for setting up fragment calculations, which need to:
- get the fragment dictionary for the input file and general setup information
- setup the fragment directories and position files inside a given directory using the output of the previous step
- edit the dictionary coming from a minBasis.yaml file so that a) the SFs point to those in the original template directory and b) for triplet claculations the SFs generated in a restricted calculation are duplicated to allow an unrestricted calculation

In [25]:
def get_fragment_dict(system_composition, xc, active_basis_size, active_rloc, env_basis_size, env_rloc,
                      active_names, env_names, active_structures, env_structures):
    # the actual dictionary to be used for the BigDFT input
    frag_dict = {}
    # unique fragments with their radical, name and position files
    frag_setup = []
    for m,molecule_dict in enumerate(system_composition):
        mol_name = molecule_dict['name']
        mol_type = molecule_dict['type']
        if mol_type == 'active':
            frag_name = get_radical_name(mol_name, xc, active_basis_size, active_rloc)
            structure = active_structures[active_names.index(mol_name)]
        elif mol_type == 'environment':
            frag_name = get_radical_name(mol_name, xc, env_basis_size, env_rloc)
            structure = env_structures[env_names.index(mol_name)]
        else:
            print('Error with fragment type ', system_composition[molecule], ' must be active or environment')
            assert False
    
        if frag_name in frag_dict:
            frag_dict[frag_name].append(m + 1)
        else:
            frag_dict[frag_name] = [m + 1]
            frag_setup.append({frag_name: {'name': mol_name, 'radical': frag_name, 'structure': structure}})

    return frag_dict, frag_setup

@RemoteFunction
def setup_fragment_directory(dirname, frag_setup, remote_dir):
    from os.path import exists, join, islink
    from os import mkdir, symlink
    from BigDFT.IO import write_xyz
    import yaml
    
    if not exists(dirname):
        mkdir(dirname)

    # use absolute paths to avoid need to change directory or use ../
    for f,frag_dict in enumerate(frag_setup):
        for frag_name in frag_dict:
            frag_data_dir_src = join(remote_dir, 'data-'+frag_setup[f][frag_name]['radical'])
            frag_data_dir_dest = join(remote_dir, dirname, 'data-'+frag_setup[f][frag_name]['radical'])
            minbasis_yaml_dest = '../../data-'+frag_setup[f][frag_name]['radical']+'/'
    
            # write out the template xyz files directly so that we have the correct environment tags
            frag_xyz_dest = join(remote_dir, dirname, frag_setup[f][frag_name]['radical']+'.xyz')
            ofile = open(frag_xyz_dest, 'w')
            write_xyz(frag_setup[f][frag_name]['structure'], ofile)
            ofile.close()
            
            # new approach
            # first make the folder
            if not exists(frag_data_dir_dest):
                mkdir(frag_data_dir_dest)
                
            # then read in and modify the minBasis.yaml file
            minbasis_filename = join(frag_data_dir_src, 'minBasis.yaml')
            with open(minbasis_filename, 'r') as ifile:
                minbasis_dict = yaml.load(ifile, Loader=yaml.Loader)

            if 'T1' in dirname:
                duplicate_spin = True
            else:
                duplicate_spin = False
            new_minbasis_dict = modify_minbasis_yaml(minbasis_dict, minbasis_yaml_dest,
                                                     duplicate_spin=False)
    
            with open(join(frag_data_dir_dest, 'minBasis.yaml'), 'w+') as ofile:
                yaml.dump(new_minbasis_dict, ofile)
                
            # finally make symbolic links for the coeff and kernel files
            # these shouldn't be overwritten as system calculations output to the top level data dir only
            for file in ['minBasis_coeff', 'density_kernel_sparse.mtx']:
                if not islink(join(frag_data_dir_dest, file)):
                    symlink(join(frag_data_dir_src, file), join(frag_data_dir_dest, file))
    
            
@RemoteFunction
def modify_minbasis_yaml(input_dict, template_dir, duplicate_spin=False):
    from copy import deepcopy
    
    output_dict = deepcopy(input_dict)
    
    # add the template dir path to each SF name
    for locreg in output_dict['functions']:
        locreg['function'] = template_dir + locreg['function']
        
    # if needed, duplicate the spin    
    if duplicate_spin:
        # a list for storing the locregs associated with spin down
        locreg_downs = []
        for locreg in output_dict['functions']:
            locreg['spin'] = 'up'
            locreg_down = deepcopy(locreg)
            locreg_down['spin'] = 'down'
            locreg_downs.append(locreg_down)

        output_dict['functions'] += locreg_downs
        
    return output_dict

The first function can be run locally to generate the fragment dictionary for each system, which is independent of the constraint and excitation type

In [26]:
fragment_dicts = {}
fragment_setup = {}
for system in system_compositions:
    fragment_dicts[system], fragment_setup[system] = get_fragment_dict(system_compositions[system], xc,
                                                                       active_basis_size, active_rloc,
                                                                       env_basis_size, env_rloc,
                                                                       active_names, env_names,
                                                                       active_structures, env_structures) 

Generate a list of all calculations to be run, including pure and mixed constraints

In [27]:
# function to identify the active molecule in a system
# assumes that there is only one active molecule
def get_active_molecule(system_composition):
    for m,molecule_dict in enumerate(system_composition):
        if molecule_dict['type'] == 'active':
            return {'name': molecule_dict['name'], 'index': m}
    
calc_list = {}
for system in system_compositions:
    # add the S0 calculation
    calc_list[system] = [get_system_name(system, xc, 'S0')]
    
    # now add the excited state pure and mixed constraints
    active_molecule = get_active_molecule(system_compositions[system])['name']
    mol_index = active_names.index(active_molecule)
    for calc_type in ['S1']:
        for component in active_tddft_runs.results[mol_index]['components'][state]:
            occ_orb = component[0]
            virt_orb = component[1]
            orbital_names = [orbital_index_to_name(occ_orb, occupied=True),
                             orbital_index_to_name(virt_orb, occupied=False)]
            calc_list[system].append(get_system_name(system, xc, calc_type, orbitals=orbital_names))
        # add the mixed calculation - to keep things general we assume that all calculations will be mixed
        calc_list[system].append(get_system_name(system, xc, calc_type, mixed=True))

    print('List of calculations for '+system+': ', calc_list[system])

List of calculations for acetone:  ['acetone_PBE_SF_S0', 'acetone_PBE_SF_S1_pure_HOMO_LUMO', 'acetone_PBE_SF_S1_mixed']
List of calculations for acetone_water:  ['acetone_water_PBE_SF_S0', 'acetone_water_PBE_SF_S1_pure_HOMO_LUMO', 'acetone_water_PBE_SF_S1_mixed']
List of calculations for naphthalene:  ['naphthalene_PBE_SF_S0', 'naphthalene_PBE_SF_S1_pure_HOMO-2_LUMO+2', 'naphthalene_PBE_SF_S1_pure_HOMO-1_LUMO+1', 'naphthalene_PBE_SF_S1_pure_HOMO_LUMO', 'naphthalene_PBE_SF_S1_mixed']
List of calculations for naphthalene_dimer:  ['naphthalene_dimer_PBE_SF_S0', 'naphthalene_dimer_PBE_SF_S1_pure_HOMO-2_LUMO+2', 'naphthalene_dimer_PBE_SF_S1_pure_HOMO-1_LUMO+1', 'naphthalene_dimer_PBE_SF_S1_pure_HOMO_LUMO', 'naphthalene_dimer_PBE_SF_S1_mixed']


Now we can run the second function remotely on the front end using jupyter magic, looping over all calculations in the above list

In [28]:
%load_ext remotemanager

In [29]:
%%sanzu url=url, remote_dir=join(remote_dir, calc_dir), serialiser=serialdill(), skip=False, avoid_nodes=True
%%sanzu extra="""source /work/e89/e89/lra040/bigdft-suite/Build/install/bin/bigdftvars.sh"""
%%sargs system_compositions=system_compositions, calc_list=calc_list
%%sargs fragment_setup=fragment_setup, remote_dir=remote_dir, calc_dir=calc_dir 
%%sargs active_structures=active_structures, env_structures=env_structures

from os.path import join

for system in system_compositions:
    for calc_name in calc_list[system]:    
        dirname = 'data-'+calc_name
        setup_fragment_directory(dirname, fragment_setup[system], join(remote_dir, calc_dir),
                                 active_structures, env_structures)

appended run runner-0
assessing run for runner dataset-b8efa681-runner-0... running


### Step 5: Run System Calculations for S0 and S1 with Pure Constraints

Define a function which will run the system calculations

In [30]:
def run_system_calculation(system, calc_name, inp):
    from BigDFT import Calculators as C
    from os.path import join
    
    Ha2eV = 27.211396132

    # create the calculator
    code = C.SystemCalculator(skip=True)
    
    # now run the code
    run = code.run(input=inp, sys=system, name=calc_name) 

    # put this as a dictionary so we could eventually add whether or not the calculation converged
    return {'energy': Ha2eV * run.energy}

Define a function which will modify the input file to add a constraint

In [31]:
def add_constraint(inp, state, orbitals, frag_index, ihomo, Vc=-500.0):
    from BigDFT import CDFT
    
    if state == 'S1':     
        constraint = CDFT.OpticalConstraint(kind='SINGLET')
        inp['dft'].update({'nspin': 1})

    elif state == 'T1':
        constraint = CDFT.OpticalConstraint(kind='TRIPLET')
        inp['dft'].update({'nspin': 2})
            
    constraint.append_hole_component(fragment=frag_index, orbital=orbitals[0], coefficient=1.0)
    constraint.append_electron_component(fragment=frag_index, orbital=orbitals[1], coefficient=1.0)
    constraint.set_Vc(Vc)
    inp.add_cdft_constraint(constraint, homo_per_fragment={1: ihomo})    
    inp['lin_general'].update({'kernel_restart_mode': 'diag'})

Set up and submit the dataset

In [33]:
run_args = {}
run_args['mpi'] = nnodes * nmpi
run_args['omp'] = nomp
run_args['walltime'] = 1 * 3600

pure_tcdft_runs = Dataset(function=run_system_calculation, url=url, name='run_system_calculation',
                          remote_dir=join(remote_dir, calc_dir), local_dir=calc_dir, serialiser=serialdill(),
                          skip=False)

pure_tcdft_run_names = []

for s,system in enumerate(system_compositions):  
    for c,calc_name in enumerate(calc_list[system]):
        inps = get_LS_input_file(hgrid, crmult, frmult, xc, active_basis_size, active_rloc, input_type='system',
                                 env_basis_size=env_basis_size, env_rloc=env_rloc, env_tag=env_tag)
        
        # make sure we remove any constraints for the S0 calculation
        # we can also restart from kernel, no need to do the more robust diagonal kernel guess as in T-CDFT
        if 'S0' in calc_name:
            if 'constrained_dft' in inps:
                inps.pop('constrained_dft', None)
                inps['lin_general'].update({'kernel_restart_mode': 'kernel'})
            
        # skip any mixed calculations, as we will come back to them later
        elif 'mixed' in calc_name:
            continue
            
        # extract the orbitals from the calculation name and update the input file for any pure calculations
        else:
            orbital_str = calc_name.split('pure', 1)[1]
            occ_orb = orbital_str.split('_', 1)[1]
            virt_orb = occ_orb.split('_', 1)[1]
            occ_orb = occ_orb.split('_', 1)[0]
            if 'S1' in calc_name:
                state = 'S1'
            elif 'T1' in calc_name:
                state = 'T1'
            
            # update the input file for any pure calculations
            active_mol = get_active_molecule(system_compositions[system])
            ihomo = active_sf_runs.results[active_names.index(active_mol['name'])]['ihomo']
            add_constraint(inps, state, [occ_orb, virt_orb], active_mol['index']+1, ihomo, Vc=-500.0)
            
            # also make sure we write out the kernel so we can do the mixed constraint
            inps['lin_general'].update({'output_mat': 1})

        # add fragment dictionary to the input file
        inps.update({'frag': fragment_dicts[system]})
    
        run_args['jobname'] = calc_name
        # this relies on consistent ordering when iterating over system_compositions,
        # which I think should be the case, but maybe we want to make this more robust/explicit
        func_kwargs = dict(system=system_structures[s], calc_name=calc_name, inp=inps)
    
        pure_tcdft_runs.append_run(args=func_kwargs, local_dir=calc_dir, **run_args)
    
        pure_tcdft_run_names.append(calc_name)

pure_tcdft_runs.run()

appended run runner-0
appended run runner-1
appended run runner-2
appended run runner-3
appended run runner-4
appended run runner-5
appended run runner-6
appended run runner-7
appended run runner-8
appended run runner-9
appended run runner-10
appended run runner-11
assessing run for runner run_system_calculation-7fb6df27-runner-0... running
128 total cores requested
appending partition = qos (standard)
script parsing complete
assessing run for runner run_system_calculation-7fb6df27-runner-1... running
128 total cores requested
appending partition = qos (standard)
script parsing complete
assessing run for runner run_system_calculation-7fb6df27-runner-2... running
128 total cores requested
appending partition = qos (standard)
script parsing complete
assessing run for runner run_system_calculation-7fb6df27-runner-3... running
128 total cores requested
appending partition = qos (standard)
script parsing complete
assessing run for runner run_system_calculation-7fb6df27-runner-4... running
1

Pause to allow calculations to finish if running the notebook using "run all"

In [34]:
assert False

AssertionError: 

Check status

In [35]:
print(pure_tcdft_runs.is_finished)

[True, True, True, True, True, True, True, True, True, True, True, True]


Fetch results

In [36]:
pure_tcdft_runs.fetch_results()

Summarize results

In [37]:
def get_pure_tcdft_table(system_name, calc_list, tcdft_results):
    
    column_names = ['Occ. Orb.', 'Virt. Orb.', 'Energy (eV)']
    row_names = [system_name]
    
    table_data = [[''] * 3] 
    c = 0
    for calc_name in calc_list:
        # get the S0 reference energy
        if 'S0' in calc_name:
            S0_energy = tcdft_results[c]['energy']
            c += 1
            continue
        elif 'mixed' in calc_name:
            continue
        else:
            orbital_str = calc_name.split('pure', 1)[1]
            occ_orb = orbital_str.split('_', 1)[1]
            virt_orb = occ_orb.split('_', 1)[1]
            occ_orb = occ_orb.split('_', 1)[0]

            table_data.append([occ_orb, virt_orb, '{0:.2f}'.format(tcdft_results[c]['energy'] - S0_energy)])
            if 'S1' in calc_name:
                row_names.append('S1')
            elif 'T1' in calc_name:
                row_names.append('T1')
            c += 1 

    tcdft_table = pd.DataFrame(table_data, index=row_names, columns=column_names)
    return tcdft_table


# summarise the pure T-CDFT results in a single table for each system
ind = 0
for s,system in enumerate(system_compositions): 
    # number of calculations which are not mixed
    ncalcs = 0
    for calc_name in calc_list[system]:
        if 'mixed' not in calc_name:
            ncalcs += 1
    pure_tcdft_table = get_pure_tcdft_table(system, calc_list[system], pure_tcdft_runs.results[ind:ind+ncalcs])
    display(pure_tcdft_table)
    ind += ncalcs

,Occ. Orb.,Virt. Orb.,Energy (eV)
acetone,,,
S1,HOMO,LUMO,5.40


,Occ. Orb.,Virt. Orb.,Energy (eV)
acetone_water,,,
S1,HOMO,LUMO,5.38


,Occ. Orb.,Virt. Orb.,Energy (eV)
naphthalene,,,
S1,HOMO-2,LUMO+2,7.33
S1,HOMO-1,LUMO+1,6.14
S1,HOMO,LUMO,4.36


,Occ. Orb.,Virt. Orb.,Energy (eV)
naphthalene_dimer,,,
S1,HOMO-2,LUMO+2,7.36
S1,HOMO-1,LUMO+1,6.17
S1,HOMO,LUMO,4.32


### Step 6: Combine Density Kernels

Define remote function for doing the kernel combination

In [38]:
@RemoteFunction
def combine_density_kernels(kernel_info):
    from scipy.io import mmread, mmwrite
    from copy import deepcopy
    from os.path import join
    
    # combine the density kernels from calculations with a pure constraint
    for p,kernel_dir in enumerate(kernel_info['kernel_dirs']):
        kernel_file = join(kernel_dir, 'density_kernel_sparse.mtx')
        rho_a_p = mmread(kernel_file)

        # set the kernel to zero on the first step
        if p == 0:
            rho_a = deepcopy(rho_a_p)
            rho_a.data[:] = 0.0

        rho_a += kernel_info['transition_weights'][p] * rho_a_p

    mmwrite(join(kernel_info['target_dir'], 'density_kernel_sparse.mtx'), rho_a)

Generate the list of directories and weightings to use, which can then be passed to the remote function

In [39]:
kernel_combination_info = {}

for system in system_compositions:
    kernel_combination_info[system] = {'S1': {'kernel_dirs': [], 'transition_weights': []}, 
                                       'T1': {'kernel_dirs': [], 'transition_weights': []}}
    s1_ind = 0
    t1_ind = 0
    
    active_mol_name = get_active_molecule(system_compositions[system])['name']
    tddft_components = active_tddft_runs.results[active_names.index(active_mol_name)]['components']
    
    for calc_name in calc_list[system]: 
        if 'pure' in calc_name:
            if 'S1' in calc_name:
                kernel_combination_info[system]['S1']['kernel_dirs'].append('data-'+calc_name)
                kernel_combination_info[system]['S1']['transition_weights'].append(
                                                                            tddft_components['S1'][s1_ind][2])
                s1_ind += 1
            elif 'T1' in calc_name:
                kernel_combination_info[system]['T1']['kernel_dirs'].append('data-'+calc_name)
                kernel_combination_info[system]['T1']['transition_weights'].append(
                                                                            tddft_components['T1'][t1_ind][2])
                t1_ind += 1
        elif 'mixed' in calc_name:
            if 'S1' in calc_name:
                kernel_combination_info[system]['S1']['target_dir'] = 'data-'+calc_name
            elif 'T1' in calc_name:
                kernel_combination_info[system]['T1']['target_dir'] = 'data-'+calc_name
                                                 
    
    for state in ['S1']:
        # the transition weights from TDDFT are not guaranteed to be normalised 
        # (due to imposing a threshold below which we neglect contributions)
        # therefore we want to explicitly normalise the weights
        coeff_sum = sum(kernel_combination_info[system][state]['transition_weights'])
        kernel_combination_info[system][state]['transition_weights'] =\
                            [c / coeff_sum for c in kernel_combination_info[system][state]['transition_weights']]
                                                 
        print('Kernel info for '+system+' '+state+': ', kernel_combination_info[system]['S1'])  

Kernel info for acetone S1:  {'kernel_dirs': ['data-acetone_PBE_SF_S1_pure_HOMO_LUMO'], 'transition_weights': [1.0], 'target_dir': 'data-acetone_PBE_SF_S1_mixed'}
Kernel info for acetone_water S1:  {'kernel_dirs': ['data-acetone_water_PBE_SF_S1_pure_HOMO_LUMO'], 'transition_weights': [1.0], 'target_dir': 'data-acetone_water_PBE_SF_S1_mixed'}
Kernel info for naphthalene S1:  {'kernel_dirs': ['data-naphthalene_PBE_SF_S1_pure_HOMO-2_LUMO+2', 'data-naphthalene_PBE_SF_S1_pure_HOMO-1_LUMO+1', 'data-naphthalene_PBE_SF_S1_pure_HOMO_LUMO'], 'transition_weights': [0.03597399954496393, 0.03285220156600908, 0.9311737988890271], 'target_dir': 'data-naphthalene_PBE_SF_S1_mixed'}
Kernel info for naphthalene_dimer S1:  {'kernel_dirs': ['data-naphthalene_dimer_PBE_SF_S1_pure_HOMO-2_LUMO+2', 'data-naphthalene_dimer_PBE_SF_S1_pure_HOMO-1_LUMO+1', 'data-naphthalene_dimer_PBE_SF_S1_pure_HOMO_LUMO'], 'transition_weights': [0.03597399954496393, 0.03285220156600908, 0.9311737988890271], 'target_dir': 'data-na

Now run the function on the front end

In [40]:
%%sanzu url=url, remote_dir=join(remote_dir, calc_dir), skip=False, avoid_nodes=True
%%sanzu extra="""source /work/e89/e89/lra040/bigdft-suite/Build/install/bin/bigdftvars.sh"""
%%sargs system_compositions=system_compositions, kernel_combination_info=kernel_combination_info

for system in system_compositions:   
    combine_density_kernels(kernel_combination_info[system]['S1'])

appended run runner-0
assessing run for runner dataset-34085642-runner-0... running


### Step 7: Run Mixed T-CDFT Calculations

We first need to define a new system function, since mixed kernel calculations extract the energy from the logfile differently

In [41]:
def run_mixed_tcdft_calculation(system, calc_name, inp):
    from BigDFT import Calculators as C
    from os.path import join
    
    Ha2eV = 27.211396132

    # create the calculator
    code = C.SystemCalculator(skip=True)
    
    # now run the code
    run = code.run(input=inp, sys=system, name=calc_name) 
    
    e = run.log['Ground State Optimization'][4]['support function optimization'][1]['Omega']['ENERGY']

    # put this as a dictionary so we could eventually add whether or not the calculation converged
    return {'energy': Ha2eV * e}

Now we have everything needed to run the mixed calculations

In [42]:
run_args = {}
run_args['mpi'] = nnodes * nmpi
run_args['omp'] = nomp
run_args['walltime'] = 1 * 3600

mixed_tcdft_runs = Dataset(function=run_mixed_tcdft_calculation, url=url, name='run_mixed_tcdft_calculation',
                           remote_dir=join(remote_dir, calc_dir), local_dir=calc_dir, serialiser=serialdill(),
                           skip=False)

mixed_tcdft_run_names = []

for s,system in enumerate(system_compositions):  
    for c,calc_name in enumerate(calc_list[system]):
        # we already ran everything which is not a mixed calculation
        if 'mixed' not in calc_name:
            continue
        
        inpm = get_LS_input_file(hgrid, crmult, frmult, xc, active_basis_size, active_rloc, input_type='system',
                                 env_basis_size=env_basis_size, env_rloc=env_rloc, env_tag=env_tag)
            
        # modify the input to run without any SCF iterations
        # here for technical reasons we set nit = 1 (not 0), otherwise the resulting logfile is not valid
        inpm['lin_kernel'].update({'nit': 1})

        # set output_mat to 1 to force it to use ntpoly (may not be needed if the default behaviour has changed)
        inpm['lin_general'].update({'kernel_restart_mode': 'full_kernel', 'output_mat': 1})

        # add fragment dictionary to the input file
        inpm.update({'frag': fragment_dicts[system]})
        
        if 'T1' in calc_name:
            inpm['dft'].update({'nspin': 2})
        else:
            inpm['dft'].update({'nspin': 1})
    
        run_args['jobname'] = calc_name
        func_kwargs = dict(system=system_structures[s], calc_name=calc_name, inp=inpm)
    
        mixed_tcdft_runs.append_run(args=func_kwargs, local_dir=calc_dir, **run_args)
    
        mixed_tcdft_run_names.append(calc_name)

mixed_tcdft_runs.run()

appended run runner-0
appended run runner-1
appended run runner-2
appended run runner-3
assessing run for runner run_mixed_tcdft_calculation-c0557b95-runner-0... running
128 total cores requested
appending partition = qos (standard)
script parsing complete
assessing run for runner run_mixed_tcdft_calculation-c0557b95-runner-1... running
128 total cores requested
appending partition = qos (standard)
script parsing complete
assessing run for runner run_mixed_tcdft_calculation-c0557b95-runner-2... running
128 total cores requested
appending partition = qos (standard)
script parsing complete
assessing run for runner run_mixed_tcdft_calculation-c0557b95-runner-3... running
128 total cores requested
appending partition = qos (standard)
script parsing complete


Pause to allow calculations to finish if running the notebook using "run all"

In [43]:
assert False

AssertionError: 

Check status

In [44]:
print(mixed_tcdft_runs.is_finished)

[True, True, True, True]


Fetch results

In [45]:
mixed_tcdft_runs.fetch_results()

Summarize results

In [46]:
def get_mixed_tcdft_table(system_name, calc_list, pure_tcdft_results, mixed_tcdft_results,
                          kernel_combination_info):
    
    column_names = ['Weight', 'Occ. Orb.', 'Virt. Orb.', 'Energy (eV)']
    row_names = [system_name]
    
    table_data = [[''] * 4] 
    
    for state in ['S1']:
        row_names.append(state)
        table_data.append([''] * 4)
        
        indm = 0
        indp = 0
        indps = 0
        for calc_name in calc_list:            
            # get the S0 reference energy
            if 'S0' in calc_name:
                S0_energy = pure_tcdft_results[indp]['energy']
                indp += 1
                continue
            elif 'pure' in calc_name:
                if state not in calc_name:
                    indp += 1
                    continue
                orbital_str = calc_name.split('pure', 1)[1]
                occ_orb = orbital_str.split('_', 1)[1]
                virt_orb = occ_orb.split('_', 1)[1]
                occ_orb = occ_orb.split('_', 1)[0]
                weight = kernel_combination_info[state]['transition_weights'][indps]
                table_data.append(['{0:.3f}'.format(weight), occ_orb, virt_orb,
                                   '{0:.2f}'.format(pure_tcdft_results[indp]['energy'] - S0_energy)])
                row_names.append('Pure component '+str(indps + 1))
                indp += 1
                indps += 1
            elif 'mixed' in calc_name:
                if state not in calc_name:
                    indm += 1
                    continue
                table_data.append(['', '', '', '{0:.2f}'.format(mixed_tcdft_results[indm]['energy'] - S0_energy)])
                row_names.append('Mixed')
                indm += 1
                
    tcdft_table = pd.DataFrame(table_data, index=row_names, columns=column_names)
    return tcdft_table


# summarise the T-CDFT results in a single table for each system
indp = 0
indm = 0
for s,system in enumerate(system_compositions): 
    # number of calculations in each dataset associated with this system
    ncalcsp = 0
    ncalcsm = 0
    for calc_name in calc_list[system]:
        if 'mixed' not in calc_name:
            ncalcsp += 1
        else:
            ncalcsm += 1
    pure_tcdft_table = get_mixed_tcdft_table(system, calc_list[system], pure_tcdft_runs.results[indp:indp+ncalcsp],
                                             mixed_tcdft_runs.results[indm:indm+ncalcsm],
                                             kernel_combination_info[system])
    display(pure_tcdft_table)
    indp += ncalcsp
    indm += ncalcsm

,Weight,Occ. Orb.,Virt. Orb.,Energy (eV)
acetone,,,,
S1,,,,
Pure component 1,1.000,HOMO,LUMO,5.40
Mixed,,,,5.40


,Weight,Occ. Orb.,Virt. Orb.,Energy (eV)
acetone_water,,,,
S1,,,,
Pure component 1,1.000,HOMO,LUMO,5.38
Mixed,,,,5.38


,Weight,Occ. Orb.,Virt. Orb.,Energy (eV)
naphthalene,,,,
S1,,,,
Pure component 1,0.036,HOMO-2,LUMO+2,7.33
Pure component 2,0.033,HOMO-1,LUMO+1,6.14
Pure component 3,0.931,HOMO,LUMO,4.36
Mixed,,,,4.44


,Weight,Occ. Orb.,Virt. Orb.,Energy (eV)
naphthalene_dimer,,,,
S1,,,,
Pure component 1,0.036,HOMO-2,LUMO+2,7.36
Pure component 2,0.033,HOMO-1,LUMO+1,6.17
Pure component 3,0.931,HOMO,LUMO,4.32
Mixed,,,,4.41
